Hello Fellow Kagglers,

This notebook demonstrates the data processing and training process in Tensorflow.

I am excited about this competition, because my Master Thesis was on sign language recognition.

**Data Processing**

Only lips, hands and arm pose coordinates are used.

A custom Tensorflow layer handles the data processing. In short, it filters all frames without coordinates for the hands and downsamples the input to 32 frames if it is too long.

**Model**

A transformer based model is used. The embedding layer makes an ambedding per landmark(lips/left hand/right hand/arm pose) and merges these embedding with fully connected layers. The transformer consists of just 2 blocks with a simple mean pooling and fully connected layers for classification.


**V2**

* Learnable attention weights for each landmark
* Removed layer normalisation in embedding to prevent double layer normalisation at the end of embedding and start of transformer
* Removed additional fully connected layer in head before classification layer

**V3**

* Using all data for training
* Increased final embedding size 384 -> 512
* Added 10% dropout in classification layer
* Increased number of epoch 50 -> 100
* Number of transformer heads 8 -> 4

If you have any feedback or questions, please feel free to leave a comment.

Expect updates in the coming weeks!

In [ ]:
# !pip install -q tensorflow_addons
# !pip install -q wandb
# !pip install -q pyarrow
# !pip install -q fastparquet

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn
import datetime

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

from layers.PreprocessLayer import PreprocessLayer
from utils.Utils import print_shape_dtype, pd_read_s3_parquet, upload_file 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy
import boto3
import io
import wandb
import json

2023-04-11 01:15:41.778760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 01:15:43.468553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow versi

# Config

In [3]:
with open("./config/config.json") as fp:
    config = json.load(fp)

In [ ]:
# Setup Weights and Biases
wandb.login()

In [ ]:
wandb.init(project='w251-asl-fp', 
           config=config,
          sync_tensorboard=True)

In [4]:
USE_VAL = True

DIM_NAMES = ['x', 'y']
SEED = 42

# Samples

In [6]:
s3_client = boto3.client(
    "s3"
)

In [7]:
AWS_S3_BUCKET = "w251-asl-data"
TRAIN_CSV_FILE = "raw-data/train.csv"

In [ ]:
data_version = config["DATA_VERSION"]

X = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{data_version}/X.npy')
X = np.load(io.BytesIO(X['Body'].read()))

y = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{data_version}/y.npy')
y = np.load(io.BytesIO(y['Body'].read()))

NON_EMPTY_FRAME_IDXS = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{data_version}/NON_EMPTY_FRAME_IDXS.npy')
NON_EMPTY_FRAME_IDXS = np.load(io.BytesIO(NON_EMPTY_FRAME_IDXS['Body'].read()))
    
print_shape_dtype([X, y, NON_EMPTY_FRAME_IDXS], ['X', 'y', 'NON_EMPTY_FRAME_IDXS'])
print(f'# NaN Values X: {np.isnan(X).sum()}')

In [ ]:
# def get_sample(X, y, NON_EMPTY_FRAME_IDXS, batch_size=config["BATCH_SIZE"]): #TODO: Change this in config.
    
#     # Arrays to store batch in
#     X_batch = np.zeros([batch_size, config["INPUT_SIZE"], config["N_COLS"], config["N_DIMS"]], dtype=np.float32)
#     y_batch = np.arange(0, batch_size, dtype=np.int32)
#     non_empty_frame_idxs_batch = np.zeros([batch_size, config["INPUT_SIZE"]], dtype=np.float32)
    
#     # Dictionary mapping ordinally encoded sign to corresponding sample indices
#     CLASS2IDXS = {}
#     for i in range(config["NUM_CLASSES"]):
#         CLASS2IDXS[i] = np.argwhere(y == i).squeeze().astype(np.int32)
            
#     while True:
#         # Fill batch arrays
#         for i in range(config["NUM_CLASSES"]):
#             idxs = np.random.choice(CLASS2IDXS[i], n)
#             X_batch[i*n:(i+1)*n] = X[idxs]
#             non_empty_frame_idxs_batch[i*n:(i+1)*n] = NON_EMPTY_FRAME_IDXS[idxs]
        
#         yield { 'frames': X_batch, 'non_empty_frame_idxs': non_empty_frame_idxs_batch }, y_batch

In [ ]:
with tf.device('CPU'):
    train_dataset = tf.data.Dataset.from_tensor_slices((X, NON_EMPTY_FRAME_IDXS, y))

In [ ]:
# # LIPS
# LIPS_MEAN_X = np.zeros([LIPS_IDXS.size], dtype=np.float32)
# LIPS_MEAN_Y = np.zeros([LIPS_IDXS.size], dtype=np.float32)
# LIPS_STD_X = np.zeros([LIPS_IDXS.size], dtype=np.float32)
# LIPS_STD_Y = np.zeros([LIPS_IDXS.size], dtype=np.float32)

# fig, axes = plt.subplots(3, 1, figsize=(15, config["N_DIMS"]*6))
   
# for col, ll in enumerate(tqdm( np.transpose(X[:,:,LIPS_IDXS], [2,3,0,1]).reshape([LIPS_IDXS.size, config["N_DIMS"], -1]) )):
#     for dim, l in enumerate(ll):
#         v = l[np.nonzero(l)]
#         if dim == 0: # X
#             LIPS_MEAN_X[col] = v.mean()
#             LIPS_STD_X[col] = v.std()
#         if dim == 1: # Y
#             LIPS_MEAN_Y[col] = v.mean()
#             LIPS_STD_Y[col] = v.std()
        
#         axes[dim].boxplot(v, notch=False, showfliers=False, positions=[col], whis=[5,95])
        
# for ax, dim_name in zip(axes, DIM_NAMES):
#     ax.set_title(f'Lips {dim_name.upper()} Dimension', size=24)
#     ax.tick_params(axis='x', labelsize=8)
#     ax.grid(axis='y')

# plt.subplots_adjust(hspace=0.50)
# plt.show()

# LIPS_MEAN = np.array([LIPS_MEAN_X, LIPS_MEAN_Y]).T
# LIPS_STD = np.array([LIPS_STD_X, LIPS_STD_Y]).T

In [ ]:
# # LEFT HAND
# LEFT_HANDS_MEAN_X = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
# LEFT_HANDS_MEAN_Y = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
# LEFT_HANDS_STD_X = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
# LEFT_HANDS_STD_Y = np.zeros([LEFT_HAND_IDXS.size], dtype=np.float32)
# # RIGHT HAND
# RIGHT_HANDS_MEAN_X = np.zeros([RIGHT_HAND_IDXS.size], dtype=np.float32)
# RIGHT_HANDS_MEAN_Y = np.zeros([RIGHT_HAND_IDXS.size], dtype=np.float32)
# RIGHT_HANDS_STD_X = np.zeros([RIGHT_HAND_IDXS.size], dtype=np.float32)
# RIGHT_HANDS_STD_Y = np.zeros([RIGHT_HAND_IDXS.size], dtype=np.float32)

# fig, axes = plt.subplots(3, 1, figsize=(15, config["N_DIMS"]*6))
   
# for col, ll in enumerate(tqdm( np.transpose(X[:,:,HAND_IDXS], [2,3,0,1]).reshape([HAND_IDXS.size, config["N_DIMS"], -1]) )):
#     for dim, l in enumerate(ll):
#         v = l[np.nonzero(l)]
#         if dim == 0: # X
#             if col < RIGHT_HAND_IDXS.size: # LEFT HAND
#                 LEFT_HANDS_MEAN_X[col] = v.mean()
#                 LEFT_HANDS_STD_X[col] = v.std()
#             else:
#                 RIGHT_HANDS_MEAN_X[col - LEFT_HAND_IDXS.size] = v.mean()
#                 RIGHT_HANDS_STD_X[col - LEFT_HAND_IDXS.size] = v.std()
#         if dim == 1: # Y
#             if col < RIGHT_HAND_IDXS.size: # LEFT HAND
#                 LEFT_HANDS_MEAN_Y[col] = v.mean()
#                 LEFT_HANDS_STD_Y[col] = v.std()
#             else: # RIGHT HAND
#                 RIGHT_HANDS_MEAN_Y[col - LEFT_HAND_IDXS.size] = v.mean()
#                 RIGHT_HANDS_STD_Y[col - LEFT_HAND_IDXS.size] = v.std()
        
#         axes[dim].boxplot(v, notch=False, showfliers=False, positions=[col], whis=[5,95])
        
# for ax, dim_name in zip(axes, DIM_NAMES):
#     ax.set_title(f'Hands {dim_name.upper()} Dimension', size=24)
#     ax.tick_params(axis='x', labelsize=8)
#     ax.grid(axis='y')

# plt.subplots_adjust(hspace=0.50)
# plt.show()

# LEFT_HANDS_MEAN = np.array([LEFT_HANDS_MEAN_X, LEFT_HANDS_MEAN_Y]).T
# LEFT_HANDS_STD = np.array([LEFT_HANDS_STD_X, LEFT_HANDS_STD_Y]).T
# RIGHT_HANDS_MEAN = np.array([RIGHT_HANDS_MEAN_X, RIGHT_HANDS_MEAN_Y]).T
# RIGHT_HANDS_STD = np.array([RIGHT_HANDS_STD_X, RIGHT_HANDS_STD_Y]).T

In [ ]:
# # POSE
# POSE_MEAN_X = np.zeros([POSE_IDXS.size], dtype=np.float32)
# POSE_MEAN_Y = np.zeros([POSE_IDXS.size], dtype=np.float32)
# POSE_STD_X = np.zeros([POSE_IDXS.size], dtype=np.float32)
# POSE_STD_Y = np.zeros([POSE_IDXS.size], dtype=np.float32)

# fig, axes = plt.subplots(3, 1, figsize=(15, config["N_DIMS"]*6))
   
# for col, ll in enumerate(tqdm( np.transpose(X[:,:,POSE_IDXS], [2,3,0,1]).reshape([POSE_IDXS.size, config["N_DIMS"], -1]) )):
#     for dim, l in enumerate(ll):
#         v = l[np.nonzero(l)]
#         if dim == 0: # X
#             POSE_MEAN_X[col] = v.mean()
#             POSE_STD_X[col] = v.std()
#         if dim == 1: # Y
#             POSE_MEAN_Y[col] = v.mean()
#             POSE_STD_Y[col] = v.std()
        
#         axes[dim].boxplot(v, notch=False, showfliers=False, positions=[col], whis=[5,95])
        
# for ax, dim_name in zip(axes, DIM_NAMES):
#     ax.set_title(f'Pose {dim_name.upper()} Dimension', size=24)
#     ax.tick_params(axis='x', labelsize=8)
#     ax.grid(axis='y')

# plt.subplots_adjust(hspace=0.50)
# plt.show()

# POSE_MEAN = np.array([POSE_MEAN_X, POSE_MEAN_Y]).T
# POSE_STD = np.array([POSE_STD_X, POSE_STD_Y]).T

# Model Config

In [ ]:
# Epsilon value for layer normalisation
LAYER_NORM_EPS = 1e-6

# Dense layer units for landmarks
LIPS_UNITS = 384
HANDS_UNITS = 384
POSE_UNITS = 384
# final embedding and transformer embedding size
UNITS = 512

# Transformer
NUM_BLOCKS = 2
MLP_RATIO = 2

# Dropout
EMBEDDING_DROPOUT = 0.00
MLP_DROPOUT_RATIO = 0.30
CLASSIFIER_DROPOUT_RATIO = 0.10

# Initiailizers
# INIT_HE_UNIFORM = tf.keras.initializers.he_uniform
# INIT_GLOROT_UNIFORM = tf.keras.initializers.glorot_uniform
# INIT_ZEROS = tf.keras.initializers.constant(0.0)
# Activations
ACTIVATION = tf.keras.activations.gelu

VERBOSE = True

In [ ]:
def get_model():
    # Inputs
    frames = tf.keras.layers.Input([config["INPUT_SIZE"], N_COLS, config["N_DIMS"]], dtype=tf.float32, name='frames')
    non_empty_frame_idxs = tf.keras.layers.Input([config["INPUT_SIZE"]], dtype=tf.float32, name='non_empty_frame_idxs')
    # Padding Mask
    mask = tf.cast(tf.math.not_equal(non_empty_frame_idxs, -1), tf.float32)
    mask = tf.expand_dims(mask, axis=2)
    
    """
        left_hand: 468:489
        pose: 489:522
        right_hand: 522:543
    """
    x = frames
    x = tf.slice(x, [0,0,0,0], [-1,config["INPUT_SIZE"], N_COLS, 2])
    # LIPS
    lips = tf.slice(x, [0,0,LIPS_START,0], [-1,config["INPUT_SIZE"], 40, 2])
    lips = tf.where(
            tf.math.equal(lips, 0.0),
            0.0,
            (lips - LIPS_MEAN) / LIPS_STD,
        )
    lips = tf.reshape(lips, [-1, config["INPUT_SIZE"], 40*2])
    # LEFT HAND
    left_hand = tf.slice(x, [0,0,40,0], [-1,config["INPUT_SIZE"], 21, 2])
    left_hand = tf.where(
            tf.math.equal(left_hand, 0.0),
            0.0,
            (left_hand - LEFT_HANDS_MEAN) / LEFT_HANDS_STD,
        )
    left_hand = tf.reshape(left_hand, [-1, config["INPUT_SIZE"], 21*2])
    # RIGHT HAND
    right_hand = tf.slice(x, [0,0,61,0], [-1,config["INPUT_SIZE"], 21, 2])
    right_hand = tf.where(
            tf.math.equal(right_hand, 0.0),
            0.0,
            (right_hand - RIGHT_HANDS_MEAN) / RIGHT_HANDS_STD,
        )
    right_hand = tf.reshape(right_hand, [-1, config["INPUT_SIZE"], 21*2])
    # POSE
    pose = tf.slice(x, [0,0,82,0], [-1,config["INPUT_SIZE"], 10, 2])
    pose = tf.where(
            tf.math.equal(pose, 0.0),
            0.0,
            (pose - POSE_MEAN) / POSE_STD,
        )
    pose = tf.reshape(pose, [-1, config["INPUT_SIZE"], 10*2])
    
    x = lips, left_hand, right_hand, pose
        
    print(x)
    x = Embedding()(lips, left_hand, right_hand, pose, non_empty_frame_idxs)
    
    # Encoder Transformer Blocks
    x = Transformer(NUM_BLOCKS)(x, mask)
    
    # Pooling
    x = tf.reduce_sum(x * mask, axis=1) / tf.reduce_sum(mask, axis=1)
    # Classifier Dropout
    x = tf.keras.layers.Dropout(CLASSIFIER_DROPOUT_RATIO)(x)
    # Classification Layer
    x = tf.keras.layers.Dense(config["NUM_CLASSES"], activation=tf.keras.activations.softmax, kernel_initializer=INIT_GLOROT_UNIFORM)(x)
    
    outputs = x
    
    # Create Tensorflow Model
    model = tf.keras.models.Model(inputs=[frames, non_empty_frame_idxs], outputs=outputs)
    
    # Simple Categorical Crossentropy Loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    
    # Adam Optimizer with weight decay
    optimizer = tfa.optimizers.AdamW(learning_rate=config["LEARNING_RATE"], weight_decay=config["WEIGHT_DECAY"])
    
    # TopK Metrics
    metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name='top_10_acc'),
    ]
    
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return model

In [ ]:
tf.keras.backend.clear_session()

model = get_model()

In [ ]:
model.predict(dummy_dataset, verbose=VERBOSE, steps=1).flatten()

In [ ]:
# Plot model summary
model.summary(expand_nested=True)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, show_layer_names=True, expand_nested=True, show_layer_activations=True)

# Weight Initialization

In [ ]:
N = 32
y_pred = model.predict(dummy_dataset, verbose=VERBOSE, steps=N).flatten()

plt.figure(figsize=(12,5))
plt.title(f'Softmax Output Initialized Model | µ={y_pred.mean():.3f}, σ={y_pred.std():.3f}', pad=25)
pd.Series(y_pred).plot(kind='hist', bins=128, label='Class Probability')
plt.xlim(0, max(y_pred) * 1.1)
plt.vlines([1 / config["NUM_CLASSES"]], 0, plt.ylim()[1], color='red', label=f'Random Guessing Baseline 1/config["NUM_CLASSES"]={1 / config["NUM_CLASSES"]:.3f}')
plt.grid()
plt.legend()
plt.show()

# Learning Rate Scheduler

In [ ]:
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=config["N_EPOCHS"]):
    
    if current_step < num_warmup_steps:
        if WARMUP_METHOD == 'log':
            return lr_max * 0.10 ** (num_warmup_steps - current_step)
        else:
            return lr_max * 2 ** -(num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

In [ ]:
def plot_lr_schedule(lr_schedule, epochs):
    fig = plt.figure(figsize=(20, 10))
    plt.plot([None] + lr_schedule + [None])
    # X Labels
    x = np.arange(1, epochs + 1)
    x_axis_labels = [i if epochs <= 40 or i % 5 == 0 or i == 1 else None for i in range(1, epochs + 1)]
    plt.xlim([1, epochs])
    plt.xticks(x, x_axis_labels) # set tick step to 1 and let x axis start at 1
    
    # Increase y-limit for better readability
    plt.ylim([0, max(lr_schedule) * 1.1])
    
    # Title
    schedule_info = f'start: {lr_schedule[0]:.1E}, max: {max(lr_schedule):.1E}, final: {lr_schedule[-1]:.1E}'
    plt.title(f'Step Learning Rate Schedule, {schedule_info}', size=18, pad=12)
    
    # Plot Learning Rates
    for x, val in enumerate(lr_schedule):
        if epochs <= 40 or x % 5 == 0 or x is epochs - 1:
            if x < len(lr_schedule) - 1:
                if lr_schedule[x - 1] < val:
                    ha = 'right'
                else:
                    ha = 'left'
            elif x == 0:
                ha = 'right'
            else:
                ha = 'left'
            plt.plot(x + 1, val, 'o', color='black');
            offset_y = (max(lr_schedule) - min(lr_schedule)) * 0.02
            plt.annotate(f'{val:.1E}', xy=(x + 1, val + offset_y), size=12, ha=ha)
    
    plt.xlabel('Epoch', size=16, labelpad=5)
    plt.ylabel('Learning Rate', size=16, labelpad=5)
    plt.grid()
    plt.show()

# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=config["N_WARMUP_EPOCHS"], lr_max=config["config["LEARNING_RATE"]"], num_cycles=0.50) for step in range(config["N_EPOCHS"])]
# Plot Learning Rate Schedule
plot_lr_schedule(LR_SCHEDULE, epochs=config["N_EPOCHS"])
# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=1)

# Weight Decay Callback

In [ ]:
# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, config["WD_RATIO"]=config["WD_RATIO"]):
        self.step_counter = 0
        self.config["WD_RATIO"] = config["WD_RATIO"]
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.config["config["LEARNING_RATE"]"] * self.config["WD_RATIO"]
        print(f'learning rate: {model.optimizer.config["config["LEARNING_RATE"]"].numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

# Performance Benchmark

In [ ]:
%%timeit -n 100
# Verify model prediction is <<<100ms
model.predict_on_batch({ 'frames': X[:1], 'non_empty_frame_idxs': NON_EMPTY_FRAME_IDXS[:1] })
pass

# Training

In [ ]:
if USE_VAL:
    # Split data based on participant id
    splitter = GroupShuffleSplit(test_size=0.10, n_splits=2, random_state=SEED)

    train_file = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=TRAIN_CSV_FILE)
    PARTICIPANT_IDS = pd.read_csv(train_file.get("Body"))['participant_id']

    train_idxs, val_idxs = next(splitter.split(X, y, groups=PARTICIPANT_IDS))

    X_train = X[train_idxs]
    X_val = X[val_idxs]
    NON_EMPTY_FRAME_IDXS_TRAIN = NON_EMPTY_FRAME_IDXS[train_idxs]
    NON_EMPTY_FRAME_IDXS_VAL = NON_EMPTY_FRAME_IDXS[val_idxs]
    y_train = y[train_idxs]
    y_val = y[val_idxs]
    
    # Define validation Data
    validation_data = ({ 'frames': X_val, 'non_empty_frame_idxs': NON_EMPTY_FRAME_IDXS_VAL }, y_val)

    print(f'Patient ID Intersection Train/Val: {set(PARTICIPANT_IDS[train_idxs]).intersection(PARTICIPANT_IDS[val_idxs])}')
    print(f'X_train shape: {X_train.shape}, X_val shape: {X_val.shape}')
    print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}')
else:
    X_train = X
    NON_EMPTY_FRAME_IDXS_TRAIN = NON_EMPTY_FRAME_IDXS
    y_train = y
    validation_data = None

In [ ]:
if USE_VAL:
    # Verify Validation Dataset Covers All Signs
    print(f'# Unique Signs in Validation Set: {pd.Series(y_val).nunique()}')
    # Value Counts
    display(pd.Series(y_val).value_counts().to_frame('Count').iloc[[1,2,3,-3,-2,-1]])

# Evaluate Initialzied Model

In [ ]:
# Sanity Check
_ = model.evaluate(*validation_data, verbose=2)

# Train

In [ ]:
# Clear all models in GPU
tf.keras.backend.clear_session()

# Get new fresh model
model = get_model()

# Sanity Check
model.summary()

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Actual Training
history = model.fit(
        x=get_train_batch_all_signs(X_train, y_train, NON_EMPTY_FRAME_IDXS_TRAIN),
        steps_per_epoch=len(X_train) // (config["NUM_CLASSES"] * config["BATCH_ALL_SIGNS_N"]),
        epochs=config["N_EPOCHS"],
        # Only used for validation data since training data is a generator
        config["BATCH_SIZE_VAL"]=config["BATCH_SIZE_VAL"],
        validation_data=validation_data,
        callbacks=[
            lr_callback,
            WeightDecayCallback(),
            tensorboard_callback
        ],
        verbose = VERBOSE,
    )

In [ ]:
# Save Model Weights
# model.save_weights(f'tf_models/{version}_model.h5')

s3_client.upload_file(Filename=f'tf_models/{version}_model.h5',
                  Bucket=AWS_S3_BUCKET,
                  Key=f'tf_models/{version}_model.h5')

In [ ]:
# Dictionaries to translate sign <-> ordinal encoded sign
train_file = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=TRAIN_CSV_FILE)
train = pd.read_csv(train_file.get("Body"))

# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [ ]:
if USE_VAL:
    # Validation Predictions
    y_val_pred = model.predict({ 'frames': X_val, 'non_empty_frame_idxs': NON_EMPTY_FRAME_IDXS_VAL }, verbose=2).argmax(axis=1)
    # Label
    labels = [ORD2SIGN.get(i).replace(' ', '_') for i in range(config["NUM_CLASSES"])]

# Landmark Attention Weights

In [ ]:
# Landmark Weights
weights = scipy.special.softmax(model.get_layer('embedding').weights[15])
landmarks = ['lips_embedding', 'left_hand_embedding', 'right_hand_embedding', 'pose_embedding']

# Learned attention weights, initialized at uniform 25%
for w, lm in zip(weights, landmarks):
    print(f'{lm} weight: {(w*100):.1f}%')

# Classification Report

In [ ]:
def print_classification_report():
    # Classification report for all signs
    classification_report = sklearn.metrics.classification_report(
            y_val,
            y_val_pred,
            target_names=labels,
            output_dict=True,
        )
    # Round Data for better readability
    classification_report = pd.DataFrame(classification_report).T
    classification_report = classification_report.round(2)
    classification_report = classification_report.astype({
            'support': np.uint16,
        })
    # Add signs
    classification_report['sign'] = [e if e in SIGN2ORD else -1 for e in classification_report.index]
    classification_report['sign_ord'] = classification_report['sign'].apply(SIGN2ORD.get).fillna(-1).astype(np.int16)
    # Sort on F1-score
    classification_report = pd.concat((
        classification_report.head(config["NUM_CLASSES"]).sort_values('f1-score', ascending=False),
        classification_report.tail(3),
    ))

    pd.options.display.max_rows = 999
    display(classification_report)

In [ ]:
if USE_VAL:
    print_classification_report()

# Training History

In [ ]:
def plot_history_metric(metric, f_best=np.argmax, ylim=None, yscale=None, yticks=None):
    plt.figure(figsize=(20, 10))
    
    values = history.history[metric]
    config["N_EPOCHS"] = len(values)
    val = 'val' in ''.join(history.history.keys())
    # Epoch Ticks
    if config["N_EPOCHS"] <= 20:
        x = np.arange(1, config["N_EPOCHS"] + 1)
    else:
        x = [1, 5] + [10 + 5 * idx for idx in range((config["N_EPOCHS"] - 10) // 5 + 1)]

    x_ticks = np.arange(1, config["N_EPOCHS"]+1)

    # Validation
    if val:
        val_values = history.history[f'val_{metric}']
        val_argmin = f_best(val_values)
        plt.plot(x_ticks, val_values, label=f'val')

    # summarize history for accuracy
    plt.plot(x_ticks, values, label=f'train')
    argmin = f_best(values)
    plt.scatter(argmin + 1, values[argmin], color='red', s=75, marker='o', label=f'train_best')
    if val:
        plt.scatter(val_argmin + 1, val_values[val_argmin], color='purple', s=75, marker='o', label=f'val_best')

    plt.title(f'Model {metric}', fontsize=24, pad=10)
    plt.ylabel(metric, fontsize=20, labelpad=10)

    if ylim:
        plt.ylim(ylim)

    if yscale is not None:
        plt.yscale(yscale)
        
    if yticks is not None:
        plt.yticks(yticks, fontsize=16)

    plt.xlabel('epoch', fontsize=20, labelpad=10)        
    plt.tick_params(axis='x', labelsize=8)
    plt.xticks(x, fontsize=16) # set tick step to 1 and let x axis start at 1
    plt.yticks(fontsize=16)
    
    plt.legend(prop={'size': 10})
    plt.grid()
    plt.show()

In [ ]:
plot_history_metric('loss', f_best=np.argmin)

In [ ]:
plot_history_metric('acc', ylim=[0,1], yticks=np.arange(0.0, 1.1, 0.1))

In [ ]:
plot_history_metric('top_5_acc', ylim=[0,1], yticks=np.arange(0.0, 1.1, 0.1))

In [ ]:
plot_history_metric('top_10_acc', ylim=[0,1], yticks=np.arange(0.0, 1.1, 0.1))

# Submission

Submission code loosley based on [this notebook](https://www.kaggle.com/code/dschettler8845/gislr-learn-eda-baseline#baseline) by [Darien Schettler
](https://www.kaggle.com/dschettler8845)

In [ ]:
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y']
    data = pd_read_s3_parquet(pq_path[14:], AWS_S3_BUCKET, s3_client, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [ ]:
preprocess_layer = PreprocessLayer(config["N_ROWS"], config["N_DIMS"], HAND_IDXS0, LANDMARK_IDXS0, config["INPUT_SIZE"])

In [ ]:
# TFLite model for submission
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
        self.preprocess_layer = preprocess_layer
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, config["N_ROWS"], config["N_DIMS"]], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        # Preprocess Data
        x, non_empty_frame_idxs = self.preprocess_layer(inputs)
        # Add Batch Dimension
        x = tf.expand_dims(x, axis=0)
        non_empty_frame_idxs = tf.expand_dims(non_empty_frame_idxs, axis=0)
        # Make Prediction
        outputs = self.model({ 'frames': x, 'non_empty_frame_idxs': non_empty_frame_idxs })
        # Squeeze Output 1x250 -> 250
        outputs = tf.squeeze(outputs, axis=0)

        # Return a dictionary with the output tensor
        return {'outputs': outputs}

# Define TF Lite Model
tflite_keras_model = TFLiteModel(model)

In [ ]:
# Sanity Check
demo_raw_data = load_relevant_data_subset(train['path'].values[5])
print(f'demo_raw_data shape: {demo_raw_data.shape}, dtype: {demo_raw_data.dtype}')
demo_output = tflite_keras_model(demo_raw_data)["outputs"]
print(f'demo_output shape: {demo_output.shape}, dtype: {demo_output.dtype}')
demo_prediction = demo_output.numpy().argmax()
print(f'demo_prediction: {demo_prediction}, correct: {train.iloc[0]["sign_ord"]}')

In [ ]:
# Create Model Converter
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
# Convert Model
tflite_model = keras_model_converter.convert()
# Write Model
with open(f'tflite_models/{version}_model.tflite', 'wb') as f:
    f.write(tflite_model)
    
# Zip Model
# !zip submission.zip /kaggle/working/model.tflite

In [ ]:
# Verify TFLite model can be loaded and used for prediction
!pip install tflite-runtime
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter("tflite_models/model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=demo_raw_data)
sign = output['outputs'].argmax()

print("PRED : ", ORD2SIGN.get(sign), f'[{sign}]')
print("TRUE : ", train.sign.values[0], f'[{train.sign_ord.values[0]}]')